In [1]:
# here model is created using gamma = 0.0001 in svc other models are not converted to model.h files here
# doing them in other notebooks

In [1]:
import pandas as pd
import numpy as np #import numpy library, used for arithmetic
from sklearn.model_selection import train_test_split #mpodule that allow us to split our train and test sets
#from sklearn.svm import SVC #the machine learning classifier model
#import required ML model from scikit learn
#using scikit learn SVM for classification 
from sklearn.svm import SVC #the machine learning classifier model
from sklearn.ensemble import RandomForestClassifier

In [3]:
# problem arised we have no vertical acceleration in mpu6050 sensor so remove those 3 cols from csv

In [2]:
df = pd.read_csv('Perfect_Total_fall_nofall.csv',sep=",")  #read the csv file
df.head() #show the 5 first rows of the dataframe

,avg_x_accn,avg_y_accn,avg_z_accn,avg_x_gyro,avg_y_gyro,avg_z_gyro,avg_x_veracc,avg_y_veracc,avg_z_veracc,Target
0,-0.133,1.567,9.967,0.200,0.133,0.067,0.208,-5.679,-8.176,0
1,-0.200,1.167,10.033,-0.233,0.033,0.000,0.113,-5.409,-8.405,0
2,-0.200,0.800,10.167,-0.400,-0.100,0.033,0.082,-5.147,-8.696,0
3,-0.133,0.767,10.067,-0.433,-0.133,-0.033,0.122,-5.054,-8.647,0
4,-0.233,-0.100,9.267,-0.133,-0.067,0.000,-0.033,-3.939,-8.298,0


In [3]:
# creating new df 
df=df.drop(['avg_x_veracc', 'avg_y_veracc', 'avg_z_veracc'], axis=1)

In [4]:
df

,avg_x_accn,avg_y_accn,avg_z_accn,avg_x_gyro,avg_y_gyro,avg_z_gyro,Target
0,-0.133,1.567,9.967,0.200,0.133,0.067,0
1,-0.200,1.167,10.033,-0.233,0.033,0.000,0
2,-0.200,0.800,10.167,-0.400,-0.100,0.033,0
3,-0.133,0.767,10.067,-0.433,-0.133,-0.033,0
4,-0.233,-0.100,9.267,-0.133,-0.067,0.000,0
...,...,...,...,...,...,...,...
98457,1.575,0.200,-4.150,-0.400,-0.075,-0.075,1
98458,2.150,0.025,-4.200,-0.375,-0.050,0.050,1
98459,2.200,0.025,-4.375,-0.225,0.000,0.075,1
98460,1.600,0.075,-4.275,-0.125,0.050,0.175,1


In [5]:
df['Target'].value_counts() #this method inspect the count for each value in a column
# logistic regression only two outputs 

Target
0    49486
1    48976
Name: count, dtype: int64

In [6]:
labels = np.array(df.pop('Target')) #extract the label column and convert it into numpy ndarray
features = np.array(df) #convert our dataframe into numpy ndarray

In [7]:
labels

array([0, 0, 0, ..., 1, 1, 1])

In [8]:
features

array([[-0.133,  1.567,  9.967,  0.2  ,  0.133,  0.067],
       [-0.2  ,  1.167, 10.033, -0.233,  0.033,  0.   ],
       [-0.2  ,  0.8  , 10.167, -0.4  , -0.1  ,  0.033],
       ...,
       [ 2.2  ,  0.025, -4.375, -0.225,  0.   ,  0.075],
       [ 1.6  ,  0.075, -4.275, -0.125,  0.05 ,  0.175],
       [ 1.6  ,  0.325, -4.2  , -0.1  ,  0.025,  0.275]])

In [9]:
#Split the dataset into training set 85% and test set 15%
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.15,shuffle=True) 

In [10]:
model = SVC(gamma=0.001) #lets instanciate our model

In [11]:
model.fit(train_features, train_labels) #we train our model

SVC(gamma=0.001)

In [12]:
print("Accuracy for the model : {} %".format(100*round(model.score(test_features,test_labels),2))) #we verify the accuracy of our model

Accuracy for the model : 69.0 %


In [17]:
model_svc_auto = SVC(gamma='auto')

In [18]:
model_svc_auto.fit(train_features, train_labels) #we train our model

SVC(gamma='auto')

In [19]:
print("Accuracy for the model : {} %".format(100*round(model_svc_auto.score(test_features,test_labels),2))) #we verify the accuracy of our model

Accuracy for the model : 80.0 %


In [20]:
rf_model = RandomForestClassifier()

In [21]:
rf_model.fit(train_features, train_labels)

RandomForestClassifier()

In [22]:
print("Accuracy for the model : {} %".format(100*round(rf_model.score(test_features,test_labels),2))) #we verify the accuracy of our model

Accuracy for the model : 87.0 %


In [13]:
from micromlgen import port

In [14]:
LABELS = ['NOFALL','FALL'] #our existing labels
classMap = {} #create an empty dict
for i, label in zip(range(2),LABELS): #interate over the range and the labels at the same time 
  classMap[i]=label #fill our dict

print(classMap)

{0: 'NOFALL', 1: 'FALL'}


In [15]:
c_code = port(model,classmap=classMap) #convert our model

#Let's write it into a .h file
modelFile = open("model.h", "w")
modelFile.write(c_code)
modelFile.close()

#Let's print the size of the .h file
import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 8,186,458 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
